Note: value of $r$ is assigned in block [24]

In [13]:
import numpy as np
import torch
import sys
from utils_old import *
import utils
from matplotlib import pyplot as plt
import torch.nn as nn
import time
from numba import cuda
from tqdm import tqdm, trange
import os
import pyroc
import pandas as pd
import gc
from IPython.display import clear_output
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="4"  
device = torch.device("cuda:0")
dtype = torch.float32
torch.manual_seed(42)
np.random.seed(42)

In [14]:
H = 300
out= 100
L = 1
class DN(torch.nn.Module):
    def __init__(self, H=300, out=100):
        super(DN, self).__init__()
        self.restored = False
        self.model = torch.nn.Sequential(
            torch.nn.Linear(28, H, bias=True),
            torch.nn.ReLU(),
            torch.nn.Linear(H, H, bias=True),
            torch.nn.ReLU(),
            torch.nn.Linear(H, H, bias=True),
            torch.nn.ReLU(),
            torch.nn.Linear(H, H, bias=True),
            torch.nn.ReLU(),
            torch.nn.Linear(H, H, bias=True),
            torch.nn.ReLU(),
            torch.nn.Linear(H, out, bias=True),
        )
    def forward(self, input):
        output = self.model(input)
        return output

class another_DN(torch.nn.Module):
    def __init__(self, H=300, out=100):
        super(another_DN, self).__init__()
        self.restored = False
        self.model = torch.nn.Sequential(
            torch.nn.Linear(28, H, bias=True),
            torch.nn.Tanh(),
            torch.nn.Linear(H, H, bias=True),
            torch.nn.Tanh(),
            torch.nn.Linear(H, H, bias=True),
            torch.nn.Tanh(),
            torch.nn.Linear(H, H, bias=True),
            torch.nn.Tanh(),
            torch.nn.Linear(H, H, bias=True),
            torch.nn.Tanh(),
            torch.nn.Linear(H, 28, bias=True),
        )
    def forward(self, input):
        output = self.model(input) + input
        return output

class Classifier(torch.nn.Module):
    def __init__(self, H=300, layers = 5):
        super(Classifier, self).__init__()
        self.restored = False
        self.model = torch.nn.Sequential(
            torch.nn.Linear(28, H, bias=True),
            torch.nn.ReLU(),
            torch.nn.Linear(H, H, bias=True),
            torch.nn.ReLU(),
            torch.nn.Linear(H, H, bias=True),
            torch.nn.ReLU(),
            torch.nn.Linear(H, H, bias=True),
            torch.nn.ReLU(),
            torch.nn.Linear(H, 1, bias=True),
            torch.nn.Sigmoid(),
        )
        if layers == 6:
            self.model = torch.nn.Sequential(
                torch.nn.Linear(28, H, bias=True),
                torch.nn.ReLU(),
                torch.nn.Linear(H, H, bias=True),
                torch.nn.ReLU(),
                torch.nn.Linear(H, H, bias=True),
                torch.nn.ReLU(),
                torch.nn.Linear(H, H, bias=True),
                torch.nn.ReLU(),
                torch.nn.Linear(H, H, bias=True),
                torch.nn.ReLU(),
                torch.nn.Linear(H, 1, bias=True),
                torch.nn.Sigmoid(),
            )
    def forward(self, input):
        output = self.model(input)
        return output
dataset = np.load('HIGGS.npy')
dataset_P = dataset[dataset[:,0]==0][:, 1:] # background (5829122, 28)
dataset_Q = dataset[dataset[:,0]==1][:, 1:] # signal     (5170877, 28)
del dataset

# Creat class

In [15]:
def generate_PQ(X_test, Y_test, model, another_model, epsilonOPT, sigmaOPT, sigma0OPT, cst,
                n_test, Samples, batch_size = 5000, If_n_large_MonteCarlo = 1000, test_batch_size=20000):
    '''
    Input: 
        n_eval: n_eval in our paper, then we generate $X^{eval}$ and $Y^{eval}$ used to compute MMD.
        Samples: we generate 'Samples' samples from P and Q, then compute their scores f(sample;X_eval,Y_eval)
    Output:
        X_eval, Y_eval: generate from P and Q, each with size (n_eval, 28)
        P_scores, Q_scores: scores of samples from P and Q, each with size (Samples,)
        EKxx, EKyy, EKxy: expectation of K(X,X), K(Y,Y), K(X,Y), where X~P_X, Y~P_Y. Then the gamma in our paper can be computed from these three values.
    '''
    # X_test = dataset_P[np.random.choice(dataset_P.shape[0], n_test, replace=False)]
    # Y_test = dataset_Q[np.random.choice(dataset_Q.shape[0], n_test, replace=False)]
    EKxx, EKyy, EKxy = compute_gamma(X_test, Y_test, model, another_model, epsilonOPT, sigmaOPT, sigma0OPT, cst, MonteCarlo=If_n_large_MonteCarlo)
    batches = (Samples-1)//batch_size + 1
    P_scores = np.zeros(Samples)
    Q_scores = np.zeros(Samples)
    for i in trange(batches):
        remain = batch_size
        if i==batches-1:
            remain = Samples - batch_size*(batches-1)
        S_hand = np.concatenate((dataset_P[np.random.choice(dataset_P.shape[0], remain, replace=False)],
                    dataset_Q[np.random.choice(dataset_Q.shape[0], remain, replace=False)]), axis=0)
        S_hand = MatConvert(S_hand, device, dtype)
        PQhat_hand = compute_score_func(S_hand, X_test, Y_test, 
                    model, another_model, epsilonOPT, sigmaOPT, sigma0OPT, cst,
                    M = test_batch_size)
        PQhat_hand = PQhat_hand.cpu().detach().numpy()
        P_scores[i*batch_size: i*batch_size+remain] = PQhat_hand[:remain]
        Q_scores[i*batch_size: i*batch_size+remain] = PQhat_hand[remain:]
        del S_hand, PQhat_hand
        gc.collect()
    return X_test, Y_test, P_scores, Q_scores, EKxx, EKyy, EKxy

class PQ_data():
    '''
    Define a class to compute the p-value, the Type I/II error.
    '''
    def __init__(self, P_scores, Q_scores, EKxx, EKyy, EKxy):
        self.P_scores = P_scores
        self.Q_scores = Q_scores
        self.P_mean = np.mean(P_scores)
        self.P_std = np.std(P_scores)
        self.Q_mean = np.mean(Q_scores)
        self.Q_std = np.std(Q_scores)
        self.EKxx = EKxx
        self.EKyy = EKyy
        self.EKxy = EKxy
    def pval_T_m_in_sigma(self, pi, m, use_gaussian, MonteCarlo):
        '''
        Compute the expected significance of discovery.
        Input:
            pi: the strength of the mixture in our paper.
            m: the number of samples, i.e. the size of Z in our paper.
            use_gaussian: if use Gaussian Approximation, then use_gaussian = True, else use_gaussian = False.
            MonteCarlo: when use_gaussian = False, we use Monte Carlo to approximate the expectation.
        Output:
            p: the expected significance of discovery (in units of Gaussian sigma)
        '''
        T = pi*self.Q_mean + (1-pi)*self.P_mean
        P_scores = self.P_scores
        mean = self.P_mean
        std = self.P_std
        if m==1:
            p = np.mean(P_scores > T)
            p = -scipy.stats.norm.ppf(p)
            self.p = p
            return p
        if use_gaussian:
            p = (T-mean)/std*np.sqrt(m)
        else:
            T_mix_MonteCarlo_list = np.zeros(MonteCarlo)
            for i in range(MonteCarlo):
                idx = np.random.choice(P_scores.shape[0], m, replace=False)
                T_mix_MonteCarlo_list[i] = np.mean(P_scores[idx])
            p = np.mean(T_mix_MonteCarlo_list > T)
            p = -scipy.stats.norm.ppf(p)
        self.p = p
        return p
    def type_1_error_H0(self, pi, m, use_gaussian, MonteCarlo):
        '''
        Compute the estimated Type I error.
        Input:
            pi: the strength of the mixture in our paper.
            m: the number of samples, i.e. the size of Z in our paper.
            use_gaussian: if use Gaussian Approximation, then use_gaussian = True, else use_gaussian = False.
            MonteCarlo: when use_gaussian = False, we use Monte Carlo to approximate the expectation.
        Output:
            type_1_error: the estimated Type I error.   
        '''
        P_scores = self.P_scores
        Q_scores = self.Q_scores
        mean = self.P_mean
        std = self.P_std
        P_mean = self.P_mean
        P_std = self.P_std
        Q_mean = self.Q_mean
        gamma = self.EKxx*(pi/2-1) + self.EKxy*(1-pi) + self.EKyy*(pi/2)
        #gamma = (pi/2)*Q_mean + (1-pi/2)*P_mean
        self.gamma = gamma
        if m==1:
            type_1_error = np.mean(P_scores > gamma)
            self.type_1_error = type_1_error
            return type_1_error
        if use_gaussian:
            type_1_error = 1-scipy.stats.norm.cdf((gamma-mean)/std*np.sqrt(m))
        else:
            MonteCarlo_list = np.zeros(MonteCarlo)
            for i in range(MonteCarlo):
                idx = np.random.choice(P_scores.shape[0], m, replace=False)
                MonteCarlo_list[i] = np.mean(P_scores[idx])
            type_1_error = np.mean(MonteCarlo_list > gamma)
            del MonteCarlo_list
            gc.collect()
        self.type_1_error = type_1_error
        return type_1_error
    def type_2_error_H1(self, pi, m, use_gaussian, MonteCarlo):
        '''
        Compute the estimated Type II error.
        Input:
            pi: the strength of the mixture in our paper.
            m: the number of samples, i.e. the size of Z in our paper.
            use_gaussian: if use Gaussian Approximation, then use_gaussian = True, else use_gaussian = False.
            MonteCarlo: when use_gaussian = False, we use Monte Carlo to approximate the expectation.
        Output:
            type_2_error: the estimated Type II error.              
        '''
        P_scores = self.P_scores
        Q_scores = self.Q_scores
        P_mean = self.P_mean
        P_std = self.P_std
        Q_mean = self.Q_mean
        Q_std = self.Q_std
        gamma = self.EKxx*(pi/2-1) + self.EKxy*(1-pi) + self.EKyy*(pi/2)
        #gamma = (pi/2)*Q_mean + (1-pi/2)*P_mean
        self.gamma = gamma
        if m==1:
            type_2_error = np.mean(Q_scores < gamma)
            self.type_2_error = type_2_error
            return type_2_error
        if use_gaussian:
            mean = Q_mean*pi + P_mean*(1-pi)
            std = np.sqrt(pi*Q_std**2 + (1-pi)*P_std**2 + pi*(1-pi)*(P_mean-Q_mean)**2)
            type_2_error = scipy.stats.norm.cdf((gamma-mean)/std*np.sqrt(m))
        else:
            MonteCarlo_list = np.zeros(MonteCarlo)
            for i in range(MonteCarlo):
                Signals_idx = np.random.choice(Q_scores.shape[0], int(m*pi), replace=False)
                Backgrounds_idx = np.random.choice(P_scores.shape[0], int(m*(1-pi)), replace=False)
                MonteCarlo_list[i] = np.mean(np.concatenate((Q_scores[Signals_idx], P_scores[Backgrounds_idx])))
            type_2_error = np.mean(MonteCarlo_list < gamma)
            del MonteCarlo_list
            gc.collect()
        self.type_2_error = type_2_error
        return type_2_error

# Calculate distribution of scores for a given $n_{train}$

In [16]:
ns = np.array([2000000, 1600000, 1300000, 1000000, 700000, 400000, 200000, 100000, 50000, 30000, 10000, 6000, 3000, 1000, 600, 300])
ns = np.array([1600000, 1300000, 1000000, 700000, 400000, 200000, 100000, 50000, 30000, 10000, 8000, 6000, 4500, 3000, 2000, 1000])
pi=0.1
gc.collect()
torch. cuda. empty_cache()

For a given $r$ below, and some $n$, we use the checkpoint path = "./Res_Net/checkpoint $(n+r)$/0/"  
To get a smooth plot, one should change the value of $r$ from 0 to 9, and rerun the whole .ipynb 10 times manually.

In [25]:
r = 0 # For a given n, we use the checkpoint of the r-th trained network
repeats_per_trained_model = 50 # for each trained network, we repeat the experiment this many times
My_class_list = [[]]*repeats_per_trained_model

In [ ]:
for k in range(repeats_per_trained_model):
    for i,n in enumerate(ns):
        print(k,n)
        path = './Res_Net/checkpoint%d/0/'%(n+r)
        model = DN(300, 100).cuda()
        another_model = another_DN(300, 100).cuda()
        model,another_model,epsilonOPT,sigmaOPT,sigma0OPT,cst = load_model(model, another_model, path)

        n_train = n
        n_test = min(n,20000)
        n_eval = 20000
        idx = np.random.choice(dataset_P.shape[0]-n_train, n_eval+n_test, replace=False) + n_train
        idy = np.random.choice(dataset_Q.shape[0]-n_train, n_eval+n_test, replace=False) + n_train
        X_test = dataset_P[np.random.choice(n_train, n_test, replace=False)]
        Y_test = dataset_Q[np.random.choice(n_train, n_test, replace=False)]
        X_eval = dataset_P[idx][n_test:n_test+n_eval]
        Y_eval = dataset_Q[idy][n_test:n_test+n_eval]
        X_test = MatConvert(X_test, device=device, dtype=dtype)
        Y_test = MatConvert(Y_test, device=device, dtype=dtype)
        X_eval = MatConvert(X_eval, device=device, dtype=dtype)
        Y_eval = MatConvert(Y_eval, device=device, dtype=dtype)
        X_test_eval = X_test
        Y_test_eval = Y_test

        # p_soft = utils.get_pval_at_once(X_test, Y_test, X_test_eval, Y_test_eval, X_eval, Y_eval,
        #                     model,another_model,epsilonOPT,sigmaOPT,sigma0OPT,cst,
        #                     batch_size = 5000,
        #                     norm_or_binom=True)
        batch_size = 10000
        Samples = 20000
        If_n_large_MonteCarlo = 10
        X_test, Y_test, P_scores, Q_scores, EKxx, EKyy, EKxy = generate_PQ(X_test, Y_test, model, another_model, epsilonOPT, sigmaOPT, sigma0OPT, cst,
                                                                        n_test, Samples=20000, batch_size=5000, 
                                                                        If_n_large_MonteCarlo=10,
                                                                        test_batch_size=min(n,20000))
        My_class  = PQ_data(P_scores, Q_scores, EKxx, EKyy, EKxy)
        My_class_list[k].append(My_class)
        del My_class
        gc.collect()
        clear_output()
        print(k,n)

# Calculate error for different $m$
Note: one can adjust $ms$, the list of $m$ to calculate. But please make sure that generate_error_m_n_train.ipynb and plot_error_m_n_train.ipynb are using the same $ms$.

In [ ]:
# My_class_list100 = My_class_list
# error_mat = type1_mat+type2_mat
# Error_m_ntr = np.mean(type1_mat+type2_mat, axis=2)
# Error_m_ntr = error_mat[:,:,1]
MonteCarlo = 1000
step=5/49 # this can be changed
new_step = 3*step # this can be changed
ms = 10**np.linspace(0, 5, 50) # this can be changed
ms = ms.astype(int)
type1_mat_M = np.zeros((len(ns), len(ms), repeats_per_trained_model))
type2_mat_M = np.zeros((len(ns), len(ms), repeats_per_trained_model))
Error_m_ntr_M = np.zeros((len(ns), len(ms)))
type1_mat_G = np.zeros((len(ns), len(ms), repeats_per_trained_model))
type2_mat_G = np.zeros((len(ns), len(ms), repeats_per_trained_model))
Error_m_ntr_G = np.zeros((len(ns), len(ms)))

In [ ]:
for k in range(repeats_per_trained_model):
    for i,n in enumerate(ns):
        My_class = My_class_list[k][i]
        #My_class_list[k].append(My_class)
        MonteCarlo = 1000
        for j in range(len(ms)):
            print(i,j,k)
            m = ms[j]
            #P_m_ntr[i,j] += p_soft*11/np.sqrt(1100) * pi * np.sqrt(m)
            # type1 = My_class.type_1_error_H0(pi, m, use_gaussian=True, MonteCarlo=MonteCarlo)
            # type2 =  My_class.type_2_error_H1(pi, m, use_gaussian=True, MonteCarlo=MonteCarlo)
            # type1_mat_G[i,j,k] = type1
            # type2_mat_G[i,j,k] = type2
            # Error_m_ntr_G[i,j] += (type2 + type1)
            if m< 1000:
                type1 = My_class.type_1_error_H0(pi, m, use_gaussian=False, MonteCarlo=MonteCarlo) 
                type2 =  My_class.type_2_error_H1(pi, m, use_gaussian=False, MonteCarlo=MonteCarlo)
                type1_mat_M[i,j,k] = type1
                type2_mat_M[i,j,k] = type2
                Error_m_ntr_M[i,j] += (type2 + type1)
            else:
                type1 = My_class.type_1_error_H0(pi, m, use_gaussian=True, MonteCarlo=MonteCarlo)
                type2 =  My_class.type_2_error_H1(pi, m, use_gaussian=True, MonteCarlo=MonteCarlo)
                type1_mat_M[i,j,k] = type1
                type2_mat_M[i,j,k] = type2
                Error_m_ntr_M[i,j] += (type2 + type1)
                
        del My_class
        gc.collect()
        clear_output()
#P_m_ntr = P_m_ntr/repeats
# Error_m_ntr = Error_m_ntr/repeats
Error_m_ntr_M = Error_m_ntr_M/repeats_per_trained_model
Error_m_ntr_G = Error_m_ntr_G/repeats_per_trained_model

# Save


In [ ]:
r_str = str(r)
np.save('ns'+r_str, ns)
np.save('ms'+r_str, ms)
np.save('Error_m_ntr_M'+r_str, Error_m_ntr_M)
# np.save('Error_m_ntr_G'+r_str, Error_m_ntr_M)
# np.save('type1_mat_M'+r_str, type1_mat_M)
# np.save('type2_mat_M'+r_str, type2_mat_M)
# np.save('type1_mat_G'+r_str, type1_mat_M)
# np.save('type2_mat_G'+r_str, type2_mat_M)
 

In [ ]:
# fig = plt.figure(figsize=(5,4))
# plt.contourf(np.log10(ns), np.log10(ms), Error_m_ntr_M.T, levels=20)
# plt.colorbar()
# plt.xlabel('$lg(n)$')
# plt.ylabel('lg(m)')
# plt.title('Type I error + Type II error (this is not the formal plot)')

# #plt.suptitle(r'Fix kernel ($n_{train}=1.3e6$), Use Gaussian Approx='+str(Use_Gaussian)+', $\pi$='+str(pi))
# # plt.savefig('./paper/tradeoff_type.pdf')